In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset

df = pd.read_csv("dataset/train_processed.csv")

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.ensemble import ExtraTreesRegressor

df.head(5)
df_clear=df.drop(['Events','StoreType','AssortmentType','Date','Max_Gust_SpeedKm_h'], axis=1)
df_clear=df_clear.dropna(axis=0)
df_clear.shape

In [4]:
df.head(5)

,Unnamed: 0,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,...,Hail,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales
0,29,1000,2016-03-31,0,1,1,2,326,846,11907,...,0,0,0,0,1,0,9055,8731.333333,6875.714286,6490.517241
1,30,1000,2016-04-01,1,0,1,2,326,0,0,...,0,0,0,0,1,1,11907,9850.333333,7428.000000,6705.379310
2,31,1000,2016-04-02,0,1,0,2,326,851,11129,...,0,0,1,0,0,0,0,6987.333333,6300.857143,6425.689655
3,32,1000,2016-04-03,0,0,0,2,326,0,0,...,0,0,0,1,1,0,11129,7678.666667,7032.857143,6582.466667
4,33,1000,2016-04-04,1,0,0,2,326,0,0,...,0,0,0,0,0,1,0,3709.666667,7032.857143,6305.800000


In [5]:
df.shape

(501300, 49)

In [6]:
df_date=df['Date']
df_no_date=df.drop(['Date'], axis=1)

In [7]:
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())

In [8]:
df=df_norm
df['Date']=df_date
df.head(5)

,Unnamed: 0,StoreID,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date
0,0.000000,0.0,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,...,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31
1,0.000002,0.0,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01
2,0.000004,0.0,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02
3,0.000006,0.0,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03
4,0.000008,0.0,1.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,0.0,1.0,0.000000,0.154355,0.363920,0.362002,2016-04-04


In [9]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

In [14]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)

In [23]:
df_store=df_train_clear.loc[(df_train_clear['StoreID']==0.0)]
df_val1_store=df_val1_clear.loc[(df_val1_clear['StoreID']==0.0)]
y=df_store['NumberOfSales']
y_val1=df_val1_store['NumberOfSales']
X=df_store.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)
X_val1=df_val1_store.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)
df_norm.head(4)

,Unnamed: 0,StoreID,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date
0,0.000000,0.0,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,...,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31
1,0.000002,0.0,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01
2,0.000004,0.0,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02
3,0.000006,0.0,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03


In [24]:
df_store.shape

(519, 48)

In [38]:
forest = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=100, max_features=15)
forest.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features=15, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [39]:
X.head(5)

,Unnamed: 0,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,...,Hail,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales
0,0.000000,0.0,1.0,1.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606
1,0.000002,1.0,0.0,1.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941
2,0.000004,0.0,1.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885
3,0.000006,0.0,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.486489,...,0.0,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885
4,0.000008,1.0,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.194596,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.154355,0.363920,0.362002


In [40]:
print(forest.feature_importances_)

[5.43113275e-03 3.93827278e-02 4.53639891e-01 1.15051746e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.03034999e-03 3.37725852e-03 5.75577635e-03
 1.53722033e-03 5.03559733e-03 2.75224277e-03 2.32189894e-03
 3.76173753e-03 2.93335281e-03 3.22441489e-03 3.65407998e-03
 2.82813119e-03 2.72127459e-03 3.04088517e-03 3.12748791e-03
 3.00767949e-03 4.73648175e-03 4.07550313e-03 3.08563064e-03
 1.30463235e-03 5.69426061e-03 0.00000000e+00 0.00000000e+00
 3.53694643e-04 2.07203777e-04 5.31368174e-04 1.57289376e-06
 7.23509189e-05 1.85041376e-03 1.79934130e-01 1.80136018e-02
 1.39788935e-03 4.12635139e-02 2.26114875e-02 2.05014752e-02
 2.87499045e-02]


In [41]:
forest.score(X, y)

0.9853233659205607

In [42]:
missing_values = round(df.isnull().sum() / df.shape[0] * 100, 3)
features_with_missings = missing_values.drop(missing_values[missing_values == 0].index).sort_values(ascending=False)
features_with_missings

Series([], dtype: float64)

In [43]:
y_pred=forest.predict(X_val1)

In [44]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mse_val1=mean_squared_error(y_val1,y_pred)
print (sqrt(mse_val1))

0.030634042070025735
